# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-27-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-27-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-28 05:25:50,33.93911,67.709953,45839,1740,36295,7804.0,Afghanistan,117.752257,3.795894
1,NaN,NaN,NaN,Albania,2020-11-28 05:25:50,41.15330,20.168300,36245,771,17755,17719.0,Albania,1259.469039,2.127190
2,NaN,NaN,NaN,Algeria,2020-11-28 05:25:50,28.03390,1.659600,80168,2372,51946,25850.0,Algeria,182.818913,2.958787
3,NaN,NaN,NaN,Andorra,2020-11-28 05:25:50,42.50630,1.521800,6610,76,5710,824.0,Andorra,8554.973144,1.149773
4,NaN,NaN,NaN,Angola,2020-11-28 05:25:50,-11.20270,17.873900,15008,342,7697,6969.0,Angola,45.663840,2.278785


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,43851,44228,44443,44503,44706,44988,45280,45490,45716,45839
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,29837,30623,31459,32196,32761,33556,34300,34944,35600,36245
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,70629,71652,72755,73774,74862,75867,77000,78025,79110,80168


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1645,1650,1661,1675,1687,1695,1712,1725,1737,1740
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,646,657,672,685,699,716,735,743,753,771
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2206,2224,2236,2255,2272,2294,2309,2329,2352,2372


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,35295,35350,35370,35422,35934,35976,36122,36145,36232,36295
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,14216,14565,15055,15469,15842,16230,16666,17031,17352,17755
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,46326,46962,47581,48183,48794,49421,50070,50712,51334,51946


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2529,2554,2580,2597,2617,2634,2661,2686,2704,2716
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,7849,7933,8038,8131,8199,8269,8376,8473,8576,8603
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1137,1145,1151,1157,1160,1161,1167,1170,1170,1171


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,37,39,39,39,39,39,39,41,42,42
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,84,84,84,84,84,84,84,98,98,98
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,10,10,10,10,10,10,10,10,10,10


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
648,1001.0,Autauga,Alabama,US,2020-11-28 05:25:50,32.539527,-86.644082,2716,42,0,2674.0,"Autauga, Alabama, US",4861.372138,1.546392
685,1075.0,Lamar,Alabama,US,2020-11-28 05:25:50,33.779950,-88.096680,671,8,0,663.0,"Lamar, Alabama, US",4860.557769,1.192250
686,1077.0,Lauderdale,Alabama,US,2020-11-28 05:25:50,34.901719,-87.656247,4004,54,0,3950.0,"Lauderdale, Alabama, US",4317.958783,1.348651


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,13088821,264858,4947446
India,9351109,136200,8759969
Brazil,6238350,171974,5581436


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,13088821,264858,4947446,7876517,2020-11-28 05:25:50,37.936303,-91.379001
India,9351109,136200,8759969,454940,2020-11-28 05:25:50,23.088275,81.806127
Brazil,6238350,171974,5581436,484940,2020-11-28 05:25:50,-12.669522,-48.480493
France,2248209,51999,165493,2030717,2020-11-28 05:25:50,8.314863,-21.695174
Russia,2196691,38175,1697618,460898,2020-11-28 05:25:50,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1207243,21693,0
California,1185576,19089,0
Florida,979020,18363,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1207243,21693,0,1185550,2020-11-28 05:25:50,31.660643,-98.653069
California,1185576,19089,0,1166487,2020-11-28 05:25:50,37.843962,-120.728594
Florida,979020,18363,0,960657,2020-11-28 05:25:50,28.940755,-82.700744
Illinois,705063,12685,0,692378,2020-11-28 05:25:50,39.843618,-89.178330
New York,628375,34477,0,593898,2020-11-28 05:25:50,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,387793,7604,0
Illinois,Cook,298870,6476,0
Florida,Miami-Dade,224166,3799,0
Arizona,Maricopa,201340,3966,0
Texas,Harris,187867,2995,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,387793,7604,0,380189,2020-11-28 05:25:50,34.308284,-118.228241,6037.0
Illinois,Cook,298870,6476,0,292394,2020-11-28 05:25:50,41.841448,-87.816588,17031.0
Florida,Miami-Dade,224166,3799,0,220367,2020-11-28 05:25:50,25.611236,-80.551706,12086.0
Arizona,Maricopa,201340,3966,0,197374,2020-11-28 05:25:50,33.348359,-112.491815,4013.0
Texas,Harris,187867,2995,0,184872,2020-11-28 05:25:50,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,45490,34944,78025,6428,14821,140,1390388,129085,27865,260512,...,4988,72227,1,100817,1321,76727,10,2124,17535,9508
2020-11-26,45716,35600,79110,6534,14920,141,1399431,130870,27873,266038,...,5117,72409,1,101215,1331,78493,10,2137,17553,9623
2020-11-27,45839,36245,80168,6610,15008,141,1407277,132346,27886,270992,...,5303,72513,1,101524,1339,80429,10,2148,17569,9714


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,1725,743,2329,76,340,4,37714,2040,907,2667,...,73,606,0,880,35,665,1,611,357,274
2020-11-26,1737,753,2352,76,341,4,37941,2068,907,2773,...,74,607,0,884,35,680,1,612,357,274
2020-11-27,1740,771,2372,76,342,4,38216,2090,907,2886,...,74,607,0,888,35,689,1,614,357,275


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,36145,17031,50712,5542,7517,129,1217284,100913,25547,189059,...,3923,69477,0,95669,1153,60499,8,1474,16755,8336
2020-11-26,36232,17352,51334,5649,7617,130,1226662,103055,25570,196342,...,4021,69670,0,96072,1166,61241,8,1484,16779,8397
2020-11-27,36295,17755,51946,5710,7697,130,1235257,105149,25585,203251,...,4107,69780,0,96402,1170,61960,8,1491,16845,8470


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2529,2554,2580,2597,2617,2634,2661,2686,2704,2716
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,596,600,601,609,622,626,633,650,667,671
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,3487,3578,3653,3712,3783,3829,3872,3936,3967,4004
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1124,1152,1179,1201,1214,1224,1239,1248,1263,1273
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7331,7363,7421,7478,7504,7539,7597,7644,7718,7735


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-25,239318,29543,0,310850,150277,1160940,210630,109152,32995,49,...,347972,1196711,183902,3827,1521,226300,153906,43050,392438,30761
2020-11-26,241957,30102,0,314324,152625,1171802,216683,109152,33579,49,...,352376,1202804,183902,3827,1521,228900,153906,44180,398104,30761
2020-11-27,242874,30776,0,318638,153677,1185576,220953,112581,34170,49,...,356716,1207243,190044,4005,1521,230444,158167,45046,399526,31773


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-11-25        2686    8473    1170  1162   2855     691   1144    6112   
2020-11-26        2704    8576    1170  1170   2879     694   1153    6215   
2020-11-27        2716    8603    1171  1173   2888     694   1153    6240   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-25         1737     1008  ...    1239    307     1821      412   
2020-11-26         1764     1010  ...    1239    307     1821      412   
2020-11-27         1765     1015  ...    1268    320     1881      426   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-25           1598  1476  1016          0      395    386  
2020-11-26           1598  1476  1016          0      395    386  
2020-11-27           1774  1547  1037          0      415    396  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-25,3532,116,0,6524,2425,18968,2906,4926,760,0,...,4466,21454,834,64,23,4008,2704,695,3343,215
2020-11-26,3572,119,0,6568,2436,19025,2957,4926,761,0,...,4519,21639,834,64,23,4029,2704,712,3415,215
2020-11-27,3572,119,0,6588,2436,19089,2977,4961,763,0,...,4526,21693,849,67,23,4044,2703,712,3434,215


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-25          41      98      10   17     39      19     41     115   
2020-11-26          42      98      10   17     40      19     41     119   
2020-11-27          42      98      10   17     40      19     41     119   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-25           49       24  ...       6      7       10        1   
2020-11-26           50       24  ...       6      7       10        1   
2020-11-27           50       24  ...       6      7       10        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-25              6     2     4          0        8      1  
2020-11-26              6     2     4          0        8      1  
2020-11-27              6     2     4          0        8      1  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,0.004638,0.018776,0.013312,0.012124,0.005359,0.007194,0.006219,0.012257,0.000431,0.022779,...,0.024230,0.003362,0.0,0.003174,0.003799,0.022931,0.0,0.004730,0.003951,0.011705
2020-11-26,0.004968,0.018773,0.013906,0.016490,0.006680,0.007143,0.006504,0.013828,0.000287,0.021212,...,0.025862,0.002520,0.0,0.003948,0.007570,0.023017,0.0,0.006121,0.001027,0.012095
2020-11-27,0.002691,0.018118,0.013374,0.011631,0.005898,0.000000,0.005607,0.011278,0.000466,0.018621,...,0.036349,0.001436,0.0,0.003053,0.006011,0.024665,0.0,0.005147,0.000912,0.009457


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,0.007593,0.010884,0.008662,0.0,0.005917,0.0,0.007534,0.018981,0.0,0.034924,...,0.013889,0.003311,NaN,0.004566,0.0,0.013720,0.0,0.003284,0.0,0.00000
2020-11-26,0.006957,0.013459,0.009875,0.0,0.002941,0.0,0.006019,0.013725,0.0,0.039745,...,0.013699,0.001650,NaN,0.004545,0.0,0.022556,0.0,0.001637,0.0,0.00000
2020-11-27,0.001727,0.023904,0.008503,0.0,0.002933,0.0,0.007248,0.010638,0.0,0.040750,...,0.000000,0.000000,NaN,0.004525,0.0,0.013235,0.0,0.003268,0.0,0.00365


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,0.000637,0.021901,0.012822,0.007087,0.009807,0.007812,0.005493,0.019900,0.000313,0.035259,...,0.025085,0.003423,NaN,0.003409,0.000000,0.008165,0.0,0.004772,0.002873,0.004700
2020-11-26,0.002407,0.018848,0.012265,0.019307,0.013303,0.007752,0.007704,0.021226,0.000900,0.038522,...,0.024981,0.002778,NaN,0.004212,0.011275,0.012265,0.0,0.006784,0.001432,0.007318
2020-11-27,0.001739,0.023225,0.011922,0.010798,0.010503,0.000000,0.007007,0.020319,0.000587,0.035189,...,0.021388,0.001579,NaN,0.003435,0.003431,0.011741,0.0,0.004717,0.003933,0.008694


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-25,0.010356,0.023560,NaN,0.012976,0.013249,0.013736,0.020301,0.017450,0.010133,0.0,...,0.005944,0.012846,0.009779,0.017278,0.0,0.012157,0.019117,0.022978,0.015519,0.026770
2020-11-26,0.011027,0.018922,NaN,0.011176,0.015624,0.009356,0.028738,0.000000,0.017700,0.0,...,0.012656,0.005091,0.000000,0.000000,0.0,0.011489,0.000000,0.026249,0.014438,0.000000
2020-11-27,0.003790,0.022391,NaN,0.013725,0.006893,0.011755,0.019706,0.031415,0.017600,0.0,...,0.012316,0.003691,0.033398,0.046512,0.0,0.006745,0.027686,0.019602,0.003572,0.032899


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-25      0.009395  0.011581  0.002571  0.004322  0.011694  0.001449   
2020-11-26      0.006701  0.012156  0.000000  0.006885  0.008406  0.004342   
2020-11-27      0.004438  0.003148  0.000855  0.002564  0.003126  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-25      0.000875  0.010415  0.013419  0.012048  ...  0.016407   
2020-11-26      0.007867  0.016852  0.015544  0.001984  ...  0.000000   
2020-11-27      0.000000  0.004023  0.000567  0.004950  ...  0.023406   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-25      0.013201  0.016750  0.017284   0.022393  0.023578  0.034623   
2020-11-26      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2020-11-27      0.042345  0.032949  0.033981   0.110138  0.048103  0.020669   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-25            NaN  0.165192  0.007833  
2020-11-26            NaN  0.000000  0.000000  
2020-11-27            NaN  0.050633  0.025907  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-25,0.017281,0.000000,NaN,0.001381,0.008316,0.005620,0.016084,0.009219,0.003963,NaN,...,0.021500,0.009838,0.032178,0.000000,0.0,0.007288,0.005204,0.019062,0.021699,0.064356
2020-11-26,0.011325,0.025862,NaN,0.006744,0.004536,0.003005,0.017550,0.000000,0.001316,NaN,...,0.011867,0.008623,0.000000,0.000000,0.0,0.005240,0.000000,0.024460,0.021538,0.000000
2020-11-27,0.000000,0.000000,NaN,0.003045,0.000000,0.003364,0.006764,0.007105,0.002628,NaN,...,0.001549,0.002495,0.017986,0.046875,0.0,0.003723,-0.000370,0.000000,0.005564,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2020-11-25      0.051282  0.166667     0.0  0.0  0.083333     0.0    0.0   
2020-11-26      0.024390  0.000000     0.0  0.0  0.025641     0.0    0.0   
2020-11-27      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   

Province_State                               ... Wyoming                     \
Admin2           Calhoun  Chambers Cherokee  ...    Park    Platte Sheridan   
2020-11-25      0.017699  0.020833      0.0  ...     0.0  0.166667      0.0   
2020-11-26      0.034783  0.020408      0.0  ...     0.0  0.000000      0.0   
2020-11-27      0.000000  0.000000      0.0  ...     0.0  0.000000      0.0   

Province_State                                                              
Admin2         Sublette Sweetwater Teton Uinta Unassigned  Washakie Weston  
2020-11-25          0.0        0.0   0.0   0.0        NaN  0.142857    0.0  
2020-11-26          0.0        0.0   0.0   0.0        NaN  0.000000    0.0  
2020-11-27          0.0        0.0   0.0   0.0        NaN  0.000000    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,0.005238,0.020610,0.013893,0.010047,0.006752,0.003747,0.005385,0.009606,0.000405,0.020524,...,0.022562,0.003045,0.0,0.003349,0.003303,0.022877,6.082025e-49,0.003947,0.002580,0.010191
2020-11-26,0.005103,0.019691,0.013899,0.013269,0.006716,0.005445,0.005944,0.011717,0.000346,0.020868,...,0.024212,0.002782,0.0,0.003649,0.005436,0.022947,3.041012e-49,0.005034,0.001803,0.011143
2020-11-27,0.003897,0.018905,0.013637,0.012450,0.006307,0.002722,0.005775,0.011498,0.000406,0.019745,...,0.030281,0.002109,0.0,0.003351,0.005723,0.023806,1.520506e-49,0.005091,0.001357,0.010300


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,0.007779,0.017589,0.008126,0.000007,0.004228,0.000081,0.006735,0.016005,2.060541e-12,0.037634,...,0.012461,0.001935,NaN,0.003787,2.056075e-27,0.015669,0.0,0.001867,0.000213,0.004114
2020-11-26,0.007368,0.015524,0.009001,0.000003,0.003584,0.000041,0.006377,0.014865,1.030270e-12,0.038690,...,0.013080,0.001793,NaN,0.004166,1.028038e-27,0.019113,0.0,0.001752,0.000106,0.002057
2020-11-27,0.004547,0.019714,0.008752,0.000002,0.003258,0.000020,0.006813,0.012752,5.151352e-13,0.039720,...,0.006540,0.000896,NaN,0.004346,5.140188e-28,0.016174,0.0,0.002510,0.000053,0.002853


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-25,0.002468,0.024031,0.012924,0.010888,0.010169,0.004159,0.005995,0.024372,0.000268,0.037573,...,0.021879,0.003145,NaN,0.003699,0.001565,0.008436,3.563686e-51,0.005322,0.001874,0.003434
2020-11-26,0.002437,0.021439,0.012595,0.015098,0.011736,0.005955,0.006850,0.022799,0.000584,0.038047,...,0.023430,0.002962,NaN,0.003956,0.006420,0.010350,1.781843e-51,0.006053,0.001653,0.005376
2020-11-27,0.002088,0.022332,0.012258,0.012948,0.011119,0.002978,0.006928,0.021559,0.000585,0.036618,...,0.022409,0.002270,NaN,0.003695,0.004925,0.011045,8.909216e-52,0.005385,0.002793,0.007035


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-25,0.010138,0.021925,NaN,0.013099,0.012585,0.013493,0.020848,0.017177,0.012176,1.175989e-77,...,0.007063,0.012720,0.012780,0.018318,0.003536,0.012114,0.021860,0.022542,0.015277,0.026672
2020-11-26,0.010582,0.020423,NaN,0.012137,0.014105,0.011424,0.024793,0.008589,0.014938,5.879945e-78,...,0.009860,0.008906,0.006390,0.009159,0.001768,0.011802,0.010930,0.024395,0.014857,0.013336
2020-11-27,0.007186,0.021407,NaN,0.012931,0.010499,0.011590,0.022250,0.020002,0.016269,2.939972e-78,...,0.011088,0.006298,0.019894,0.027835,0.000884,0.009273,0.019308,0.021998,0.009215,0.023117


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-25      0.009070  0.011339  0.003170  0.012779  0.012731  0.001301   
2020-11-26      0.007886  0.011748  0.001585  0.009832  0.010569  0.002821   
2020-11-27      0.006162  0.007448  0.001220  0.006198  0.006847  0.001411   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-25      0.002797  0.009008  0.012890  0.009659  ...  0.016696   
2020-11-26      0.005332  0.012930  0.014217  0.005821  ...  0.008348   
2020-11-27      0.002666  0.008476  0.007392  0.005386  ...  0.015877   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-25      0.015929  0.025477  0.030292   0.029038  0.020900  0.032493   
2020-11-26      0.007964  0.012739  0.015146   0.014519  0.010450  0.016246   
2020-11-27      0.025155  0.022844  0.024563   0.062328  0.029276  0.018458   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-25           -1.0  0.103448  0.013081  
2020-11-26           -1.0  0.051724  0.006541  
2020-11-27           -1.0  0.051179  0.016224  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-25,0.010110,0.032600,NaN,0.003044,0.008720,0.004830,0.014000,0.006366,0.004590,NaN,...,0.017323,0.008302,0.021819,0.004925,1.651385e-08,0.006507,0.007807,0.018102,0.021671,0.051886
2020-11-26,0.010718,0.029231,NaN,0.004894,0.006628,0.003917,0.015775,0.003183,0.002953,NaN,...,0.014595,0.008463,0.010910,0.002462,8.256926e-09,0.005873,0.003904,0.021281,0.021604,0.025943
2020-11-27,0.005359,0.014616,NaN,0.003970,0.003314,0.003641,0.011269,0.005144,0.002790,NaN,...,0.008072,0.005479,0.014448,0.024669,4.128463e-09,0.004798,0.001767,0.010641,0.013584,0.012972


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-11-25      0.026135  0.083334  0.000434 -0.001505  0.042684   
2020-11-26      0.025263  0.041667  0.000217 -0.000753  0.034162   
2020-11-27      0.012631  0.020833  0.000109 -0.000376  0.017081   

Province_State                                                            ...  \
Admin2               Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2020-11-25      2.592997e-06  2.328306e-11  0.011364  0.010417  0.004167  ...   
2020-11-26      1.296498e-06  1.164153e-11  0.023073  0.015412  0.002083  ...   
2020-11-27      6.482492e-07  5.820766e-12  0.011537  0.007706  0.001042  ...   

Province_State   Wyoming                                          \
Admin2              Park    Platte  Sheridan Sublette Sweetwater   
2020-11-25      0.062581  0.108395  0.001114      0.0   0.003906   
2020-11-26      0.031291  0.054197  0.000557      0.0   0.001953   
2020-11-27      0.015645  0.027099  0.000278      0.0   0.000977   

Province_State                                                             
Admin2                 Teton         Uinta Unassigned  Washakie    Weston  
2020-11-25      1.192093e-07  1.271595e-06  -0.972376  0.071429 -0.000005  
2020-11-26      5.960464e-08  6.357974e-07  -0.972376  0.035714 -0.000002  
2020-11-27      2.980232e-08  3.178987e-07  -0.972376  0.017857 -0.000001  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201127,AK,29554,NaN,961219,NaN,totalTestsViral,990773,159.0,691.0,...,990773,0,3,22a0f24661b0b734101d39860a395bb9fa812380,0,0,0,0,0,NaN
1,20201127,AL,242874,40401.0,1365240,NaN,totalTestsViral,1567713,1526.0,24451.0,...,1608114,0,50,f9713a2c6844d2d13742105c1561466cc630d0bb,0,0,0,0,0,NaN
2,20201127,AR,153677,18773.0,1521090,NaN,totalTestsViral,1655994,1011.0,8779.0,...,1674767,0,68,99d9b152d025521e0ff3e09ab43b30920acc77b4,0,0,0,0,0,NaN
3,20201127,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,...,1988,0,0,04b01e2d54538c0e4f4be0a7f14fdc874f8e2c5b,0,0,0,0,0,NaN
4,20201127,AZ,318638,9772.0,1894651,NaN,totalTestsPeopleViral,2203517,2301.0,25296.0,...,2213289,20,133,26059291f7fb02bd3477099733c00412f37d21c4,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201127,AK,29554.0,NaN,961219.0,NaN,totalTestsViral,990773.0,159.0,691.0,...,990773,0,3,22a0f24661b0b734101d39860a395bb9fa812380,0,0,0,0,0,NaN
1,20201127,AL,242874.0,40401.0,1365240.0,NaN,totalTestsViral,1567713.0,1526.0,24451.0,...,1608114,0,50,f9713a2c6844d2d13742105c1561466cc630d0bb,0,0,0,0,0,NaN
2,20201127,AR,153677.0,18773.0,1521090.0,NaN,totalTestsViral,1655994.0,1011.0,8779.0,...,1674767,0,68,99d9b152d025521e0ff3e09ab43b30920acc77b4,0,0,0,0,0,NaN
3,20201127,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,...,1988,0,0,04b01e2d54538c0e4f4be0a7f14fdc874f8e2c5b,0,0,0,0,0,NaN
4,20201127,AZ,318638.0,9772.0,1894651.0,NaN,totalTestsPeopleViral,2203517.0,2301.0,25296.0,...,2213289,20,133,26059291f7fb02bd3477099733c00412f37d21c4,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-27,AK,29554,NaN,961219,NaN,totalTestsViral,990773,159.0,691.0,NaN,...,990773,0,3,22a0f24661b0b734101d39860a395bb9fa812380,0,0,0,0,0,NaN
2020-11-27,AL,242874,40401.0,1365240,NaN,totalTestsViral,1567713,1526.0,24451.0,NaN,...,1608114,0,50,f9713a2c6844d2d13742105c1561466cc630d0bb,0,0,0,0,0,NaN
2020-11-27,AR,153677,18773.0,1521090,NaN,totalTestsViral,1655994,1011.0,8779.0,406.0,...,1674767,0,68,99d9b152d025521e0ff3e09ab43b30920acc77b4,0,0,0,0,0,NaN
2020-11-27,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,NaN,...,1988,0,0,04b01e2d54538c0e4f4be0a7f14fdc874f8e2c5b,0,0,0,0,0,NaN
2020-11-27,AZ,318638,9772.0,1894651,NaN,totalTestsPeopleViral,2203517,2301.0,25296.0,532.0,...,2213289,20,133,26059291f7fb02bd3477099733c00412f37d21c4,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-27,AK,29554.0,NaN,961219.0,NaN,totalTestsViral,990773.0,159.0,691.0,NaN,...,990773,0,3,22a0f24661b0b734101d39860a395bb9fa812380,0,0,0,0,0,NaN
2020-11-27,AL,242874.0,40401.0,1365240.0,NaN,totalTestsViral,1567713.0,1526.0,24451.0,NaN,...,1608114,0,50,f9713a2c6844d2d13742105c1561466cc630d0bb,0,0,0,0,0,NaN
2020-11-27,AR,153677.0,18773.0,1521090.0,NaN,totalTestsViral,1655994.0,1011.0,8779.0,406.0,...,1674767,0,68,99d9b152d025521e0ff3e09ab43b30920acc77b4,0,0,0,0,0,NaN
2020-11-27,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,NaN,...,1988,0,0,04b01e2d54538c0e4f4be0a7f14fdc874f8e2c5b,0,0,0,0,0,NaN
2020-11-27,AZ,318638.0,9772.0,1894651.0,NaN,totalTestsPeopleViral,2203517.0,2301.0,25296.0,532.0,...,2213289,20,133,26059291f7fb02bd3477099733c00412f37d21c4,0,0,0,0,0,NaN
